# Elegant Exception Handling
## Eyal Trabelsi

- thanks for coming out, I am Eyal I live in Tel Aviv and work for Salesforce.
- and today I am going to talk to you about Elegent Exception Handling.

## Today Is About

- Correct and reliable code 💪


- Yet maintainable and  readable 🌺


## Not Covering 

- PEP8 💅

- Performance implications ⌛


- Parallel/Concurrent 🧵

## Restaurant Recommendation 🍔

In [3]:
! pip install typeguard rollbar returns

     |████████████████████████████████| 51kB 403kB/s eta 0:00:01
     |████████████████████████████████| 112kB 598kB/s eta 0:00:01
  Created wheel for rollbar: filename=rollbar-0.15.0-cp37-none-any.whl size=61686 sha256=689b684f42475609ff95244a51cadc17227e02dae134d39660daecef780148e2
  Stored in directory: /Users/etrabelsi/Library/Caches/pip/wheels/1c/b5/b1/290546a7a87f78d5ce22b428d159bb4a783fb256168f9b552f
Successfully built rollbar


In [ ]:
import contextlib
import logging
import pathlib
from typing import Union

import requests
from typeguard import typechecked
from bs4 import BeautifulSoup

In [4]:
def _get_recommended_restaurant(user):
    base_url = "https://en.wikipedia.org/wiki"
    return requests.get(f"{base_url}/{user}").content

def _get_user(path):
    with open(path, 'r') as json_file:
        return json.load(json_file)["user"]

def _pick_best_restaurant(restaurants):
    pass

In [5]:
def get_food_recommendation(path):
    user = _get_user(path)
    restaurants = _get_recommended_restaurant(user)
    return _pick_best_restaurant(restaurants)
    
get_food_recommendation("MY_AMAZING_FILE.json")

NameError: name 'json' is not defined


- In our snippet we are going to bring country information from wikipedia.
- As you can see the code itself is quite easy to understand


- First we bring the user from a file
- Then we bring relevant results from google
- And lastly pick the best one


![](unicorn_rainbow.png)

- We can be proud of ourself and we are basicly done


![](unicorn.png)

- Until the errors and exceptions starts emerging.
- We see here FileNotFoundException, KeyError,JSONDecodeError, AnotherExoticError and dozens of exceptions
- As developers we want our program to keep working.
- In other words we want to build a fault tolerant program
- How fault tollerrant it is really depends whether its a mobile app or a banking system.

- **Lesson 1:** We want to build a fault tolerant to a certain degree.

![](bank_vs_facebook.jpg)

- we want to build a fault tolerant (to a certain degree).
- i am gonna show you how a naive handle exception would look like

## Naive Exception Handling 👶

In [4]:
def get_food_recommendation(path):
    try:
        user = _get_user(path)
        restaurants = _get_recommended_restaurant(user)
        return _pick_best_restaurant(restaurants)
    except BaseException:
        logging.error("VERY UNINFORMATIVE INFORMATION")
        raise BaseException("VERY UNINFORMATIVE INFORMATION")

get_food_recommendation("MY_AMAZING_FILE.json")

1.
   - This is the same snippet with a naive exception handling.
   - Basicly its the same, we just catching exceptions and logging it away.


2.
   - This code is pretty bad and unfortunately this kind of code is pretty common  as well.
   - The First issue is that there are some exceptions we dont want to catch, such as KeyboardInterrupt (as we want the user to be able to kill the program) or SynatxError (as we want our code to be valid).
   - Secondly, this code is really hard to recover from, the invoker of this function can't really destinguise between the diffrent types of errors and allow to recover from certain expected issues.
   - For example, if we have flaky internet i would like to retry, but if a file is actually missing I dont.
   - So this code is not really more reliable.
  
  
3.
   - Next we will take a more pragmatic approach.
   - We will start handle relavent exceptions only.
   - We will allow to recover from certain expected issues.

![](https://www.pngitem.com/pimgs/m/4-48557_scared-emoji-png-android-shock-emoji-png-transparent.png)

## Not So Naive Exception Handling 👦


In [6]:
def get_food_recommendation(path):
    try:
        user = _get_user(path)                
    except FileNotFoundException:
        logging.error("VERY UNINFORMATIVE INFORMATION")
        raise
    except JSONDecodeError:
        logging.error("VERY UNINFORMATIVE INFORMATION")
        raise
    except KeyError:
        user = "default_user"  
    
    resurants = _get_recommended_restaurant(user)
    return _pick_best_restaurant(restaurants)
    
get_food_recommendation("MY_AMAZING_FILE.json")

NameError: name 'FileNotFoundException' is not defined

![](https://cdn.shopify.com/s/files/1/1061/1924/products/14_grande.png?v=1571606116)

- In this snippet I am focusing in exceptions that can occur from the get_country method.

- As a reminder we dont want to catch all exceptions types (SynataxError etc)  we will catch speicific exceptions and handle them

- When a file does not exists or when the file is not a valid json we  raise FileNotFoundException and JSONDecodeError and log it away

- We will reraise the same exact exception that occured instead raising a generic exception and allow the invoker to handle them diffrently.

- Lastly when we have valid file json without country key we will use France as our default country since they have the best pasteries

- Altough this code is far from pretty is much safer, we added deafault patiserie and  the invoker of this function can destinguise between the diffrent types of errors and handle them in a diffrent manner if needed.

**Lesson 2:** Catch as specific of an error as you can. 

-  Its important to note that there are some examples when you might want to catch all exceptions, For example since Web scraping. 

- Many code bases are completely dominated by error handling like our current example.

**Lesson 3:** Error handling is important, but if it obscures the logic, it’s a bad idea

- Error handling is important, but we should strives to make our job easier.
 - as the zen of python state
      "If the implementation is hard to explain, it's a bad idea."
- There are two reasons why the code became so unreadable:
    - The first one is, there are various reasons for Exceptions in python:
        - The obious one is that something exceptional happened.
        - As a control flow mechanism.
        - Can be triggered due to a bug in our code .
    - We handled our exceptions in the wrong layer
        - we need to separate the business logic from administrative logic.
        - in order to make the business logic very stand out.      

## Our Ideal Code 💯

In [6]:
def get_food_recommendation(path):
    user = _get_user(path)
    restaurants = _get_recommended_restaurant(user)
    return _pick_best_restaurant(restaurants)

get_food_recommendation("MY_AMAZING_FILE.json")

- In order to get to our ideal code we need to move all the exception handling logic from our method
- In other words we need to push the exception handling part up or down in the abstraction level

# Pushing Exception Handling Down ⏬

**This**

In [7]:
def _get_user(path):
    with open(path, 'r') as json_file:
        return json.load(json_file)["user"]

as a reminder all the exception we handled came from get_country and this how the code looked before

**Becomes**

In [8]:
def _get_user(path):
    with open(path, 'r') as json_file:
        try:
            data = json.load(json_file)
        except FileNotFoundException:
            logging.error("VERY UNINFORMATIVE INFORMATION")
            raise
        except JSONDecodeError:
            logging.error("VERY UNINFORMATIVE INFORMATION")
            raise
        try:
            user = data["user"]
        except KeyError:
            user = "default_user"        
    return user

- In our example we pushed all the exceptions to the get_country method and make it more reliable and thus keeping get_country_capital clean


**Lesson 4:** Push the exception handling down/up in the abstraction level


- In addition I will seperate key extraction from the try catch block to another try/catch block as in many cases it will make our code less nested
- as the zen of python state "Flat is better than nested".


**Lesson 5:** Split nested try catch blocks to separated try blocks

- When our json is valid but we lack the country key we are picking france as our default country.
    - In our example we could use check if the key exists as oppose of catching KeyError.


- This is the diffrence between EAFP (it’s easier to ask for forgiveness than permission) versus LBYL (Look before you leap)
    - Each has its own pros and cons (whether the thread-safty or readability)
    - but both are legitimate in python as oppose to other languages.

- But our code is still ugly, 

## A Bit Mackup 💄


**This**

In [9]:
def _get_user(path):
    with open(path, 'r') as json_file:
        try:
            data = json.load(json_file)
        except FileNotFoundException:
            logging.error("VERY UNINFORMATIVE INFORMATION")
            raise
        except JSONDecodeError:
            logging.error("VERY UNINFORMATIVE INFORMATION")
            raise
        try:
            user = data["user"]
        except KeyError:
            user = "default_user"        
    return user

**Becomes**

In [10]:
def _get_user(path):
    with open(path, 'r') as json_file:
        try:
            data = json.load(json_file)
        except (FileNotFoundException, JSONDecodeError):
            logging.error("VERY UNINFORMATIVE INFORMATION")
            raise
        else:
            user = data.get("user", "default_user")
            return user

- This is the same code with a few cosmetic changes
- First since we handle both FileNotFoundException and JSONDecodeError in the same manner they can "share the except block"
  as except clause may name multiple exceptions as a parenthesized tuple.
- Secondly we can use else  clause which occur when the try block executed and did not raise an exception.
- Thirdly, we use dictionary builtin function get which allow us to define default values.

- and the lesson here is that Frequent flows probably have clean existing solution.

**Lesson 6:** Frequent flows probably have clean existing solution.

- There is another common flow for exception handling i want to cover which is suppressing exceptions

In [11]:
def run_unstopable_animation():
    pass


## Suppressing Exceptions 🤫

**This**

In [ ]:
try:
    os.remove('somefile.pyc')
except FileNotFoundError:
    pass

In [12]:
try:
    run_unstopable_animation()
except KeyboardInterrupt:
    pass

- Another example for such flow is try except pass 

**Becomes**

In [ ]:
from contextlib import suppress

with suppress(FileNotFoundError):
    os.remove('somefile.pyc')

In [13]:
from contextlib import suppress

with suppress(KeyboardInterrupt):
    run_unstopable_animation()

which translate to with Suppress, supported from python>=3.5

## Friendly Reminder 📑

In [14]:
def _get_user(path):
    with open(path, 'r') as json_file:
        try:
            data = json.load(json_file)
        except (FileNotFoundException, JSONDecodeError):
            logging.error("VERY UNINFORMATIVE INFORMATION")
            raise
        else:
            user = data.get("user", "default_user")
            return user

- This code is much more readble and handle the exception in quite a good manner
- The problems that can emerge from this kind of code is that it doesnt YELL the intent.
- There are few ways to emphasis our intent and the type and actual message are the most cruitial ones.

**Lesson 7:** We can improve how we express our intent with the exception type and message


![](builtin_exceptions.png)


- As you can see there are tons of builtin exceptions and one of them might fit like a glove.
- They are well documented and the invoker can do some stackoverflow magic when he face them.
- In our example its seems there are no good exception type to emphasis our intent intent more.
- So either we stay with the same exceptions thrown or we should create a custome exception.
- The question is When should I create a custom exception as oppose to using builtin ones?!

## Builtin Exceptions vs Custom Exceptions ⚔



- By default we should thrive to use builtin exceptions.

- Unless We can emphasis our intent.

- Unless We can emphasis our intent (like in our example).
    - I want to be able to convey to the user why an exception is being raised
    - not just that an exception is being raised.


- Unless We need to clearly distinguish between different exceptions.

- Another reason for custom exception is if We need to clearly distinguish between different kind of exceptions. For example
    - Lets say we have ValueError and we want to recover in diffrent way between TooBig/TooSmall.
    - Another example is when Wrapping third party exceptions. I wont go into too much detail regarding this but in a nutshell
        - when we wrap third party api we minimize our dependecy on it. for example uppon recovery shouldn't have to import exceptions from your dependecies for example requests.exceptions
        - Also the users that use your library does not need/want to know about the implementation details.
        

**Lesson 8:** Custom exceptions can be usefull


In [15]:
class MaleformConfigFile(Exception):
    pass

- In our usecase both FileNotFoundException and JSONDecodeError are indication we have MaleformConfigurationFile and the invoker would want to treat them the same 
- Its important to understand the power of renaming, as it will emphasis our intent
- Another consideration is picking the right inheritance (can even have multiple inheritance).
- you can use type inheritance to be clever about how you catch exceptions.


In [16]:
def _get_user(path):
    with open(path, 'r') as json_file:
        try:
            data = json.load(json_file)
        except (FileNotFoundException, JSONDecodeError):
            logging.error("VERY INFORMATIVE INFORMATION")
            raise MaleformConfigFile from None
        else:
            user = data.get("user", "default_user")
            return user

- So instead of just re-raising the same exception we will raise MaleformConfigFile for both FileNotFoundException, JSONDecodeError
- If I want to controll the trace that of our exception we could change the cause
- There are basicly 3 strategies to change the cause eith
    - raise MaleformConfigFile
    - raise MaleformConfigFile from None
    - raise MaleformConfigFile from e

**Lesson 9:** We can improve how we express our intent with the __cause__ of the exception

- I wont go into details but will give you an example.
    
- If the code I just showed you is a library, I might want to hide internal implementation details.
- i might want to suppress hide the FileNotFoundException, JSONDecodeError info  and only show the MaleformConfigFile part of trace.
- This is done by using from None


In [17]:
def _get_recommended_restaurant(country):
    base_url = "https://en.wikipedia.org/wiki"
    resp = requests.get(f"{base_url}/{country}")
    resp.raise_for_status()
    return resp.content

- Lets go to our get_wikipedia_country_content method
- if we want to make it more robust to the flakiness environment. 
- we might want to implement some retrying mechanism

In [18]:
def _get_recommended_restaurant(country):
    base_url = "https://en.wikipedia.org/wiki"
    allowed_retries = 5
    for i in range(1, allowed_retries + 1):
        try:
            resp = requests.get(f"{base_url}/{country}")
            resp.raise_for_status()
        except (requests.ConnectionError, requests.Timeout) as e:
            if i == allowed_retries:
                raise errors.Unavailable() from e
        else:
            return resp.content

- In this snippet we are using the techniques we showed before.
- And yet this codestill unreadable
- Just a reminder We want seperate the buisness logic as much as possible from the exception handling code, this can be used by using python decorators for the retry logic
- important note retry can be handled in the request itself by [writing an adapter](https://findwork.dev/blog/advanced-usage-python-requests-timeouts-retries-hooks/), but for the example sake i wont use it. 
- we are going to use a diffrent technique

## Decorators 🎊


- Decorators extend our function capabilities beyond its core intent.


In [19]:
from functools import wraps

In [20]:
def retry(exceptions, allowed_retries=5):
    def callable(func):
        @wraps(func)
        def wrapped(*args, **kwargs):
            for i in range(allowed_retries):
                try:
                    res = func()
                except exceptions:
                    continue
                else:
                    return res       
        return wrapped
    return callable            

- In our example we can simply create a decorator and then use it to make the buisness logic and the exception handling part seprated

In [21]:
@retry(exceptions=(requests.ConnectionError, requests.Timeout))
def _get_recommended_restaurant(country):
    base_url = "https://en.wikipedia.org/wiki"
    resp = requests.get(f"{base_url}/{country}")
    resp.raise_for_status()
    return resp.content


- There are many usecases that using decorators can make our code much cleaner. For example ratelimit retry and more.
  Most of the frequent use-cases are already exists as a seperate library 
- those libraries tend to have more features and less bugs.
  For example tenacity has advance mechanisms for the intervals between retries.
  

- **Lesson 10:** Use python builtin constructs to write cleaner code


## Usefull usecases 🧠
- [ratelimit](https://github.com/tomasbasham/ratelimit)
- [Retry](https://github.com/jd/tenacity)
- [logger.catch](https://github.com/Delgan/loguru)

- The same benefits can achived with context managers

## Context Managers 🌉
- Wrap around enter and exit logic over a given resource.


In [22]:
def context_manager_example():
    engine = create_engine("...")
    Session = sessionmaker(bind=engine)

    session = Session()
    try:
        item1 = session.query(Item).get(1)
        item2 = session.query(Item).get(2)
        item1.foo = 'bar'
        item2.bar = 'foo'
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [23]:
@contextlib.contextmanager
def transactional_session(session_cls, nested=True, **kwargs):
    session = session_cls(**kwargs)
    session.begin(nested=nested)
    try:
        yield session
    except:
        session.rollback()
        raise
    else:
        session.commit()
    finally:
        session.close()

- Like decorators there are many usecases, like temporary files, transactions database connections and more

## Usefull usecases 🧠

- [Database Connections]()
- [Transactions]()
- [Temporary Files]()
- [Output Redirections]()


## sys's excepthook! 🎣


- Python has builtin hooks for various events including uncaught exceptions 

- Overiding this using sys.excepthook.

- We can achieve exactly that by overiding our own sys.excepthook.

- Uncaught exception will print traceback to STDERR before closing.

- In a production environment, this is not acceptable. 

- We want to do some graceful exit before your program's demise. For example, notify some incident system.


In [24]:
import sys
import rollbar

rollbar.init("Super Secret Token")

def rollbar_except_hook(exc_type, exc_value, traceback):
    rollbar.report_exc_info((exc_type, exc_value, traceback))
    sys.__excepthook__(exc_type, exc_value, traceback)
    
sys.excepthook = rollbar_except_hook

- The major benefit of this approach is that it  doesn't require modifying existing code.

- I am NOT suggesting you substitute your recovery try/except blocks with this method, just as a better way to crash an application.

**Lesson 11:** Python has some usefull builtin hooks.

## Usefull usecases 🧠


- [Format Diffrently](https://dev.to/joshuaschlichting/catching-every-single-exception-with-python-40o3)
- [Redirect To Incident System](https://sweetcode.io/using-rollbar-capturing-logging-python-exceptions/)
- [Multi Threading Behaviour](https://www.scrygroup.com/tutorial/2018-02-06/python-excepthook-logging/)
- [Search Stackoverflow](https://gist.github.com/WoLpH/88e3222ac57d9c3bff113ff83afddda4/) 😛😛😛

- We can format the exceptions diffrently, to provide more/less information.
- We can redirect Exceptions to an incident system like rollbar or pager-duty.
- Since threading/multiprocessing have their own unhandled exception machinery.
  that is a bit customized so no unhandled exception exists at the top level.
  we might want to overide it to support KeyboardInterupt for example.
- Search Stackoverflow for the exception that was being raise

# Silent Errors 🔇

- Does not crash code but delivers an incorrect result.


- Makes matter worse.


- Just because programmers often ignore error messages doesn’t mean the program should stop emitting them. 
- Silent errors can happen when functions return None or other value instead of raising exceptions.
- generaly it’s better for a program to fail fast and crash than to silence the error and continue running the program. 
- The bugs that inevitably happen later on will be harder to debug since they are far removed from the original cause. 

- To know if program is correct we need to have a contract

# Contract 📜

- Output/Input types/values
- Postconditions.
- Preconditions.
- Side effects.
- Invariants.

- The first attempt to handle it is assertions


In [25]:
def _get_user(path):
    assert isinstance(path, str) or isinstance(path, pathlib.PurePath)
    with open(path, 'r') as json_file:
        try:
            data = json.load(json_file)
        except (FileNotFoundException, JSONDecodeError):
            logging.error("VERY INFORMATIVE INFORMATION")
            raise MaleformConfigFile
        else:
            user = data.get("user","default_user")
            assert isinstance(country, str)
            return user

- One can check those contracts with a simple assert
- Assert has the following cons:
    -  "assert" raises the wrong type of exception and "hides" our intent. In our example we will raise AssertionError as oppose to TypeError.
    - it can be compiled away if you run Python with the -O or -OO optimization flags
- Thus in generaly i believe its better to just raise regular exception

In [26]:
@typechecked
def get_country(path: Union[str, pathlib.PurePath]) -> str:
    with open(path, 'r') as json_file:
        try:
            data = json.load(json_file)
        except (FileNotFoundException, JSONDecodeError):
            logging.error("VERY INFORMATIVE INFORMATION")
            raise MaleformConfigFile
        else:
            user = data.get("user","default_user")
            return user

- There are some existing libraries that provide these contracts checking via decorator
- In this example i show the use of typechecked decorator of typecheck package to validate the types are correct in runtime (as oppose to mypy compile time)
- there two more packages for contracts:
    - [icontract](https://github.com/Parquery/icontract)
    - [contracts](https://github.com/AndreaCensi/contracts)


- **Lesson 12:** Use contracts to protect your code from silent errors.

In [27]:
def _get_recommended_restaurant(country):
    base_url = "https://en.wikipedia.org/wiki"
    resp = requests.get(f"{base_url}/{country}")
    resp.raise_for_status()
    return resp.content

In our usecases we can still get a lot of exception
- Your network might be down, so request won't happen at all
- The server might be down
- The server might be too busy and you will face a timeout
- The server might require an authentication
- API endpoint might not exist
- You might not have enough permissions to view it
- The server might fail with an internal error while processing your request
- The server might return an invalid or corrupted response
- The server might return invalid json, so the parsing will fail


Ok, now we will hope that someone else will catch this exception and possibly handle it.

Now we just need to check where this exception is actually caught. By the way, how can we tell where exactly it will be handled? Can we navigate to this point in the code? Turns out, we can not do that.

There’s no way to tell which line of code will be executed after the exception is thrown.

We have two independent flows in our app: regular flow that goes from top to bottom and exceptional one that goes however it wants. How can we consciously read code like this?


In [28]:
import requests
from returns.result import safe
from returns.result import Result

@safe    
def _get_recommended_restaurant(country: str) -> Result['UserProfile', Exception]:
    base_url = "https://en.wikipedia.org/wiki"
    resp = requests.get(f"{base_url}/{country}")
    resp.raise_for_status()
    return resp.content

- You can use container values, that wraps actual success or error value into a thin wrapper with utility methods to work with this value. That's exactly why we have created @dry-python/returns project. So you can make your functions return something meaningful, typed, and safe.

- "Execution flow is unclear". Now it is the same as a regular business flow. From top to bottom.
             

- We use @safe for all methods that can raise an exception, it will change the return type of the function to Result[OldReturnType, Exception]
- We use Result as a container for wrapping values and errors in a simple abstraction
- We use .unwrap() to unwrap raw value from the container
- We use @pipeline to make sequences of .unwrap calls readable




- **Lesson 13:** For complicated usecases consider the functional way.

## Sensative information 🕵

In [29]:
def login(user):
    raise CommonPasswordException(f"password: {password} is too common")

- The issue here comes down to sensitive data. 

- Exceptions for internal use can contain technical details, such as user_ids or specific data that caused the crash,

- but it’s important to remember when an exception occurs, these messages will be spread far and wide, through logging, reporting, and monitoring software.

- In a world where regulation around personal data is constantly getting stricter, you can never be too careful

-  and it’s almost always possible to provide valuable error messages without compromising customer privacy.


- **Lesson 14:** Don’t use sensitive information in exception messages.



- It’s not only customer privacy you need to worry about, though. Bad actors are everywhere.

- We should never reveal weaknesses in the internal workings of your software to users.

- Yes, you should give your users feedback about what’s going on, but never, ever tell them about the internal workings of your software.



## Lessons: 👨‍🏫👩‍🏫

- **Lesson 1:** We want to build a fault tolerant (to a certain degree).

- **Lesson 2:** You shouldn't bother to handling all possible exceptions.

- **Lesson 3:** Error handling is important, but if it obscures logic, it’s a bad idea

- **Lesson 4:** Push the exception handling down/up in the abstraction level


- **Lesson 6:** Split nested try catch blocks to separated blocks


- **Lesson 7:** Frequent flows probably have clean existing solution.


- **Lesson 8:** We can improve how we express our intent with custom exceptions with the type of exception and the message

- **Lesson 9:** We can improve how we express our intent with the __cause__ of the exception

- **Lesson 10:** Use python builtin constructs to write cleaner code

- **Lesson 11:** Python has some usefull builtin hooks

- **Lesson 12:** Use contracts to protect your code from silent errors.

- **Lesson 13:** For complicated usecases consider the functional way.    

- **Lesson 14:** Don’t use sensitive information in exception messages.

![](https://i.pinimg.com/originals/b9/0a/79/b90a79b4c361d079144597d0bcdd61de.jpg)

### Additional Resources 📚

- [Intro to Exception Handling](https://www.freecodecamp.org/news/exception-handling-python/)
- [A Comprehensive Guide to Handling Exceptions in Python](https://medium.com/better-programming/a-comprehensive-guide-to-handling-exceptions-in-python-7175f0ce81f7)
- [Exceptional Exceptions - How to properly raise, handle and create them.](https://www.youtube.com/watch?v=V2fGAv2R5j8)
- [Beyond PEP 8 -- Best practices for beautiful intelligible code](https://www.youtube.com/watch?v=wf-BqAjZb8M)
- [Passing Exceptions 101 Paradigms in Error Handling](https://www.youtube.com/watch?v=BMtJbrvwlmo)
- [Custom Exceptions](https://towardsdatascience.com/how-to-define-custom-exception-classes-in-python-bfa346629bca)
- [sys.excepthook Explained](https://dev.to/joshuaschlichting/catching-every-single-exception-with-python-40o3)
- [Overiding Threading sys.excepthook](https://www.scrygroup.com/tutorial/2018-02-06/python-excepthook-logging/)

## Not to incorporate

### Recoverability 🩹


- How do i recover
   - abandonment isolate at the process level
       - what level of isolation makes sense for me
       - how can  you make sure all bad state is cleared away to retry
       
- what is recoverable:
    - network flakiness
    - database out of connection
    - disk unavailable
    - recoverable database out of connections

### Concurrent/Parallal Exception handling 🎸🎺🎻🎷
- **Multi-threading/processing** [1](https://stackoverflow.com/questions/51071378/exception-handling-in-concurrent-futures-executor-map), [2](https://docs.python.org/3/library/concurrent.futures.html)
- **Async** [1](https://stackoverflow.com/questions/30361824/asynchronous-exception-handling-in-python),[2](https://www.roguelynn.com/words/asyncio-exception-handling/), [3](https://medium.com/@yeraydiazdiaz/asyncio-coroutine-patterns-errors-and-cancellation-3bb422e961ff)

### Error codes 👾
- WITHIN a program one should always use exceptions instead of error codes. However, exceptions can't propagate beyond a program. Any time the error must leave the program you are left with error messages or error codes.

- In high-level stuff, exceptions; in low-level stuff, error codes.

- If, however, I'm writing a piece of code which I must know the behaviour of in every possible situation, then I want error codes. Otherwise I have to know every exception that can be thrown by every line in my function to know what it will do (Read The Exception That Grounded an Airline to get an idea of how tricky this is). 

- It's tedious and hard to write code that reacts appropriately to every situation (including the unhappy ones), but that's because writing error-free code is tedious and hard, not because you're passing error code

- Error codes are almost the last thing that you want to see in an API response. Generally speaking, it means one of two things — something was so wrong in your request or your handling that the API simply couldn’t parse the passed data, or the API itself has so many problems that even the most well-formed request is going to fail. In either situation, traffic comes crashing to a halt, and the process of discovering the cause and solution begins.

- That being said, errors, whether in code form or simple error response, are a bit like getting a shot — unpleasant, but incredibly useful. Error codes are probably the most useful diagnostic element in the API space, and this is surprising, given how little attention we often pay them.

- In general, the goal with error responses is to create a source of information to not only inform the user of a problem, but of the solution to that problem as well. Simply stating a problem does nothing to fix it – and the same is true of API failures.


- [1](https://docs.python.org/3.8/library/errno.html),[2](https://medium.com/vaidikkapoor/understanding-non-blocking-i-o-with-python-part-1-ec31a2e2db9b),[3](https://nordicapis.com/best-practices-api-error-handling/),[4](https://stackoverflow.com/questions/253314/conventions-for-exceptions-or-error-codes),[5](https://www.joelonsoftware.com/2003/10/13/13/)

## [Release it](https://github.com/singh4java/Books/blob/master/Release%20It!%20Design%20and%20Deploy%20Production-Ready%20Software.pdf) 📪

- If, however, I'm writing a piece of code which I must know the behaviour of in every possible situation, then I want error codes. Otherwise I have to know every exception that can be thrown by every line in my function to know what it will do (Read The Exception That Grounded an Airline to get an idea of how tricky this is). 

- It's tedious and hard to write code that reacts appropriately to every situation (including the unhappy ones), but that's because writing error-free code is tedious and hard, not because you're passing error code


- not speak on arcitecual  patterns

- you can always reboot the world by restarding every single server layer by layer thats almost always effectove but takes long time


- its like a doctor diagnosing desease, theyou could treat a patient,


- counter integration point with circuit breaker and decoupling middleware

- a cascading failure happens after something else already gone wrong. circuit breaker protect your system by avoiding calls out to the troubled integration point. using timeout ensure that you can come back from a call out to the troubled one

- We can add the traceback using exc_info=True.